In [1]:
import sys
import numpy as np
import math
import cmath
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML                                # Make the Cell width of the notebook wider
display(HTML("<style>.container { width:80% !important; }</style>"))          

In [2]:
#-------------------------------------------------------------------------------------------------------------------------------
# Title: Solving the ODE for eta and phi
#-------------------------------------------------------------------------------------------------------------------------------
# Contents: 
#-------------------------------------------------------------------------------------------------------------------------------

# Section 1.0:   Define a function that will integrate through the domain and return phi_{n+1} 

# Section 2.0:   Define a function that will calculate R and S in d2phi/dy2 + R(y) dphi/dy + S(y) phi = 0
# Section 2.1:   Allocate memory to the variables and define the constants in the ODE
# Section 2.2:   Calculate R and S in the ODE for each profile

# Section 3.0:   Define an ODE solver that will use the previous functions to solve the ODE
# Section 3.1:   Set up the grid in y, allocate memory and define the constants
# Section 3.2.0: Set up the values for the wave speed and wave number that we will loop over 
# Section 3.2.1: Print the user defined options
# Section 3.3:   Find R and S for c_0 (first choice of wave speed) and then integrate to find phi with boundary conditions
# Section 3.3.1: Begin the iteration towards a solution for wavenumber in the outer loop 
# Section 3.3.2: Solving for the first choice of wave speed 
# Section 3.4:   Iterate towards a solution for the ODE over all the wave speed increments
# Section 3.5:   Check for convergence and update the wave speed for the next loop
# Section 3.5.1: Plots for fixing k and varying c_{i}
# Section 3.5.2: Plots for fixing c_{i} and varying k
# Section 3.6:   Update the wavenumber in preperation for the next loop

# Section 4.0:   Set up for plots
# Section 4.0.1: Plots of the profiles and coefficients of the ODEs
# Section 4.1:   Plot of the growth rate
# Section 4.2:   Plot of the real part of the phase speed
# Section 4.3:   Plot the levels where the ODEs are singular

# Section 5.0:   Time taken for the code to run

In [3]:
#-------------------------------------------------------------------------------------------------------------------------------
# Section 1.0 : Define a function that will integrate through the domain and return phi_{n+1}  
#-------------------------------------------------------------------------------------------------------------------------------
def INT_REES(phi_nm1 , phi_n,  R_n , S_n , y_nm1 , y_n , y_np1):      
    import sys
    import numpy as np
    import math
    import cmath
    import matplotlib.pyplot as plt

#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# phi_nm1 = phi_{n-1} , phi_n = phi_{n} , R_n = R_{n} , S_n = S_{n} , y_nm1 = y_{n-1} , y_n = y_{n} , y_np1 = y_{n+1}
#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Finite Difference Scheme

# It is assumed that the grid spacing is uniform

# Using a centered finite difference scheme 
# phi = phi_n                                                          
# d/dy phi = (phi_np1-phi_nm1) / 2(y_np1-y_n)                                
# d^{2}/dy^{2} phi = (phi_np1-2*phi_n+phi_nm1) / (y_np1-y_n)^{2}

# phi_np1 [ 1.0/(y_np1 - y_n)^2 + P_n/(y_np1 - y_nm1) ] +  [ (- 2 phi_n + phi_nm1 )/(y_np1 - y_n)^2  -  P_n phi_nm1/(y_np1 - y_nm1) + Q_n phi_n ] = 0

    dy      = y_np1 - y_n 
    denom   = 1.0 / (dy*dy) + (R_n/(y_np1 - y_nm1))               
    num2    =  S_n * phi_n - (R_n * phi_nm1 / (y_np1 - y_nm1))
    num1    =  (-2.0 * phi_n + phi_nm1) / (dy*dy) 
    phi_np1 = - (num1 + num2) / denom

    return phi_np1

In [4]:
#------------------------------------------------------------------------------------------------------------------------------
# Section 2.0 : Define a function that will calculate R and S in d2phi/dy2 + R(y) dphi/dy + S(y) phi = 0
#------------------------------------------------------------------------------------------------------------------------------

# Calculating R and S
def R_S_Rees(y, c_n , k):   # y is an array of np.zeros, and k is the wavenumber
    import sys
    import numpy as np
    import math
    import cmath
    import matplotlib.pyplot as plt

#------------------------------------------------------------------------------------------------------------------------------
# Section 2.1 : Allocate memory to the variables and define the constants in the ODE
#------------------------------------------------------------------------------------------------------------------------------
  
    len_y = len(y)  # y=np.zeros( no. of levels + 2 , dtype =np.complex ) , see section 3.1
  
    # Allocate memory for the numpy arrays   
    R      = np.zeros( len_y, dtype =  np.complex )     
    S      = np.zeros( len_y, dtype =  np.complex )
    Rv     = np.zeros( len_y, dtype =  np.complex )
    Sv     = np.zeros( len_y, dtype =  np.complex )
    Rs     = np.zeros( len_y, dtype =  np.complex )
    Ss     = np.zeros( len_y, dtype =  np.complex )
    f      = np.zeros( len_y, dtype =  np.complex )
    B      = np.zeros( len_y, dtype =  np.complex )
    H      = np.zeros( len_y, dtype =  np.complex )
    dHdy   = np.zeros( len_y, dtype =  np.complex )
    U      = np.zeros( len_y, dtype =  np.complex )
    dUdy   = np.zeros( len_y, dtype =  np.complex )
    HQ     = np.zeros( len_y, dtype =  np.complex )
    dHQdy  = np.zeros( len_y, dtype =  np.complex )
    D      = np.zeros( len_y, dtype =  np.complex )
    dDdy   = np.zeros( len_y, dtype =  np.complex )
    F      = np.zeros( len_y, dtype =  np.complex )
    dFdy   = np.zeros( len_y, dtype =  np.complex )
    G      = np.zeros( len_y, dtype =  np.complex )
    dGdy   = np.zeros( len_y, dtype =  np.complex )
    d2Udy2 = np.zeros( len_y, dtype =  np.complex )

    # Define the constants
    beta   = 2.2*(10**-11)       
    g      = 0.0334              # Reduced gravity
    H0     = 60                  # Average thermocline depth (see Tanaka+Hibiya)
    A      = [-0.5,-0.5,-5,-0.5]
    c      = np.sqrt(g*H0)      
    C      = np.sqrt((2*beta/c))  
    #y     = C*x                 # Non-dimensional y
    i      = 5                   # Choose your profile: i = [0 (n=0), 1 (n=2), 2 (Cubic), 3 (Quintic) , 4 (Sinusoidal), 5(Cos)]
    L      = 10

#------------------------------------------------------------------------------------------------------------------------------
# Section 2.2 : Calculate R and S in the ODE for each profile
#------------------------------------------------------------------------------------------------------------------------------
    for iy in range ( len_y ) :
        if i == 0:
            f[iy]     = beta*y[iy]           
            B[iy]     = (y[iy]**2)/4.0  
            H[iy]     = A[0] * np.exp(-B[iy])                                                     
            dHdy[iy]  = - (A[0] / 2.0) * C * y[iy] * np.exp(-B[iy])                               
            U[iy]     = (g * A[0] / c) * np.exp(-B[iy])                                           
            dUdy[iy]  = -(g * A[0] / (2*c))*C*y[iy]*np.exp(-B[iy])
            HQ[iy]    = beta * (1 / C) * y[iy] + (g * A[0] / 2 * c) * C * y[iy] * np.exp(-B[iy])  
            dHQdy[iy] = beta - (g*A[0]/4*c)*(C**3)*((y[iy]**2)-2)*np.exp(-B[iy])
            D[iy]     = (k**2) * ((U[iy] - c_n)**2) - f[iy] * HQ[iy]
            dDdy[iy]  = 2 * (k**2) * dUdy[iy] * (U[iy] - c_n) - beta * HQ[iy] - f[iy] * dHQdy[iy] 
            F[iy]     = D[iy] / (g*(H0+H[iy]))                                                         
            dFdy[iy]  = dDdy[iy] / (g*(H0+H[iy])) - (D[iy]*dHdy[iy]) / (g*((H0+H[iy])**2))        
            G[iy]     = (k**2)*((U[iy]-c_n)**2 - g*(H0+H[iy]))                                    # ODE for v
            dGdy[iy]  = (k**2)*(2*dUdy[iy]*(U[iy]-c_n) - g*dHdy[iy])                              # ODE for v
            R[iy]     = (-1.0 / F[iy])*dFdy[iy]
            S[iy]     = ((F[iy] - (k**2)) + ((f[iy] * dFdy[iy] / F[iy]) - beta) / (U[iy] - c_n))
            Rv[iy]    = (-dGdy[iy]/(G[iy]*(U[iy]-c_n)))                                           # ODE for v
            Sv[iy]    = (F[iy]-(k**2))-(dHQdy[iy] - (HQ[iy]/G[iy])*dGdy[iy])/(U[iy]-c_n)          # ODE for v
            Rs[iy]    = 0
            Ss[iy]    = (dHQdy[iy])/(U[iy]-c_n) -(k**2)
            
        if i == 1:
            f[iy]     = beta*y[iy]
            B[iy]     = (y[iy]**2)/(4.0)
            H[iy]     = A[2]*((y[iy]**2)-1)*np.exp(-B[iy])
            dHdy[iy]  = -(A[2]/2.0)*C*y[iy]*((y[iy]**2)-5)*np.exp(-B[iy])
            U[iy]     = -(g*A[2]/c)*((y[iy]**2)-5)*np.exp(-B[iy])
            dUdy[iy]  = (g*A[2]/(2*c))*C*y[iy]*np.exp(-B[iy])*((y[iy]**2)-9) 
            HQ[iy]    = beta*(1/C)*y[iy] + (g*A[2]/(2*c))*C*y[iy]*np.exp(-B[iy])*((y[iy]**2)+9) 
            dHQdy[iy] = beta + (g*A[2]/2*c)*C*(2*C*(y[iy]**2)*np.exp(-B[iy])+((y[iy]**2)+9)*(-0.5*C*((y[iy]**2)-2)*np.exp(-B[iy])))
            D[iy]     = (k**2) * ((U[iy] - c_n)**2) - f[iy] * HQ[iy]
            dDdy[iy]  = 2 * (k**2) * dUdy[iy] * (U[iy] - c_n) - beta * HQ[iy] - f[iy] * dHQdy[iy] 
            F[iy]     = D[iy] / (g*(H0+H[iy]))  
            dFdy[iy]  = dDdy[iy] / (g*(H0+H[iy])) - (D[iy]*dHdy[iy]) / (g*((H0+H[iy])**2))
            G[iy]     = (k**2)*((U[iy]-c_n)**2 - g*(H0+H[iy]))                                    # ODE for v
            dGdy[iy]  = (k**2)*(2*dUdy[iy]*(U[iy]-c_n) - g*dHdy[iy])                              # ODE for v
            R[iy]     = (-1.0 / F[iy])*dFdy[iy]
            S[iy]     = ((F[iy] - (k**2)) + ((f[iy] * dFdy[iy] / F[iy]) - beta) / (U[iy] - c_n))
            Rv[iy]    = (-dGdy[iy]/(G[iy]*(U[iy]-c_n)))                                           # ODE for v
            Sv[iy]    = (F[iy]-(k**2))-(dHQdy[iy] - (HQ[iy]/G[iy])*dGdy[iy])/(U[iy]-c_n)          # ODE for v
            Rs[iy]    = 0
            Ss[iy]    = (dHQdy[iy])/(U[iy]-c_n) -(k**2)
            
        if i == 2:
            f[iy]     = beta*y[iy]
            B[iy]     = (y[iy]**2)/4.0
            H[iy]     = (y[iy]**3)*np.exp(-B[iy])
            dHdy[iy]  = 3*(y[iy]**2)*C*np.exp(-B[iy])-0.5*C*(y[iy]**4)*np.exp(-B[iy])
            U[iy]     = (g/c)*y[iy]*np.exp(-B[iy])*((y[iy]**2)-6)
            dUdy[iy]  =  (g/c)*(-0.5*C*((y[iy]**2)-2)*((y[iy]**2)-6)*np.exp(-B[iy])+2*C*(y[iy]**2)*np.exp(-B[iy]))
            HQ[iy]    = beta*(1/C)*y[iy] - (g/c)*(-0.5*C*((y[iy]**2)-2)*((y[iy]**2)-6)*np.exp(-B[iy])+2*C*(y[iy]**2)*np.exp(-B[iy]))
            dHQdy[iy] = beta - (g/c)*C*((-0.5*C*y[iy]*np.exp(-B[iy]))*(-0.5*((y[iy]**2) -2)*((y[iy]**2) -6)+2*(y[iy]**2)) + np.exp(-B[iy])*(-2*C*(y[iy]**3)+12*y[iy]*C))
            D[iy]     = (k**2) * ((U[iy] - c_n)**2) - f[iy] * HQ[iy]
            dDdy[iy]  = 2 * (k**2) * dUdy[iy] * (U[iy] - c_n) - beta * HQ[iy] - f[iy] * dHQdy[iy] 
            F[iy]     = D[iy] / (g*(H0+H[iy]))  
            dFdy[iy]  = dDdy[iy] / (g*(H0+H[iy])) - (D[iy]*dHdy[iy]) / (g*((H0+H[iy])**2))  
            G[iy]     = (k**2)*((U[iy]-c_n)**2 - g*(H0+H[iy]))                                    # ODE for v
            dGdy[iy]  = (k**2)*(2*dUdy[iy]*(U[iy]-c_n) - g*dHdy[iy])                              # ODE for v
            R[iy]     = (-1.0 / F[iy])*dFdy[iy]
            S[iy]     = ((F[iy] - (k**2)) + ((f[iy] * dFdy[iy] / F[iy]) - beta) / (U[iy] - c_n))
            Rv[iy]    = (-dGdy[iy]/(G[iy]*(U[iy]-c_n)))                                           # ODE for v
            Sv[iy]    = (F[iy]-(k**2))-(dHQdy[iy] - (HQ[iy]/G[iy])*dGdy[iy])/(U[iy]-c_n)          # ODE for v
            Rs[iy]    = 0
            Ss[iy]    = (dHQdy[iy])/(U[iy]-c_n) -(k**2)
            
        if i == 3:
            f[iy]     = beta*y[iy]
            B[iy]     = (y[iy]**2)/4.0
            H[iy]     = (y[iy]**5)*np.exp(-B[iy])
            dHdy[iy]  = -0.5*C*(y[iy]**4)*np.exp(-B[iy])*((y[iy]**2)-10)
            U[iy]     = (g/c)*(y[iy]**3)*np.exp(-B[iy])*((y[iy]**2)-10)
            dUdy[iy]  = (g/c)*(-(1/2)*C*((y[iy]**2))*((y[iy]**2)-6)*((y[iy]**2)-10)*np.exp(-B[iy])+2*C*(y[iy]**4)*np.exp(-B[iy]))
            HQ[iy]    = beta*(1/C)*y[iy] - (g/c)*(-(1/2)*C*((y[iy]**2))*((y[iy]**2)-6)*((y[iy]**2)-10)*np.exp(-B[iy])+2*C*(y[iy]**4)*np.exp(-B[iy]))
            dHQdy[iy] = beta - (g/c)*C*(-0.5*C*y[iy]*np.exp(-B[iy])*(-0.5*(y[iy]**2)*((y[iy]**2) -6)*((y[iy]**2) -10)+2*(y[iy]**4))+np.exp(-B[iy])*(6*C*(y[iy]**5)-56*C*(y[iy]**3)+120*C*y[iy]))
            D[iy]     = (k**2) * ((U[iy] - c_n)**2) - f[iy] * HQ[iy]
            dDdy[iy]  = 2 * (k**2) * dUdy[iy] * (U[iy] - c_n) - beta * HQ[iy] - f[iy] * dHQdy[iy] 
            F[iy]     = D[iy] / (g*(H0+H[iy]))
            dFdy[iy]  = dDdy[iy] / (g*(H0+H[iy])) - (D[iy]*dHdy[iy]) / (g*((H0+H[iy])**2))
            G[iy]     = (k**2)*((U[iy]-c_n)**2 - g*(H0+H[iy]))                                    # ODE for v
            dGdy[iy]  = (k**2)*(2*dUdy[iy]*(U[iy]-c_n) - g*dHdy[iy])                              # ODE for v
            R[iy]     = (-1.0 / F[iy])*dFdy[iy]
            S[iy]     = ((F[iy] - (k**2)) + ((f[iy] * dFdy[iy] / F[iy]) - beta) / (U[iy] - c_n))
            Rv[iy]    = (-dGdy[iy]/(G[iy]*(U[iy]-c_n)))                                           # ODE for v
            Sv[iy]    = (F[iy]-(k**2))-(dHQdy[iy] - (HQ[iy]/G[iy])*dGdy[iy])/(U[iy]-c_n)          # ODE for v
            Rs[iy]    = 0
            Ss[iy]    = (dHQdy[iy])/(U[iy]-c_n) -(k**2)
            
        if i == 4:
            f[iy]     = beta*y[iy]
            B[iy]     = (y[iy]**2)/(4.0)
            m         = 0
            U[iy]     = A[0]*np.sin(m*np.pi*y[iy] / (C*L))   
            H[iy]     = -(A[0]*(beta) / g)*((L/m*np.pi)**2)*((np.sin(m*np.pi*y[iy]/(C*L))-(L/(m*np.pi*C))*y[iy]*np.cos(m*np.pi*y[iy]/(C*L))))
            dUdy[iy]  = A[0]*(m*np.pi/L)*np.cos(m*np.pi*y[iy]/(C*L))
            dHdy[iy]  = -(beta/(C*g))*y[iy]*A[0]*np.sin(m*np.pi*y[iy]/(C*L))
            HQ[iy]    = (beta/C)*y[iy] - A[0]*(m*np.pi/L)*np.cos(m*np.pi*y[iy]/(C*L))
            dHQdy[iy] =  beta + A[0]*((m*np.pi/L)**2)*np.sin(m*np.pi*y[iy]/(C*L)) 
            D[iy]     = (k**2) * ((U[iy] - c_n)**2) - f[iy] * HQ[iy]
            dDdy[iy]  = 2 * (k**2) * dUdy[iy] * (U[iy] - c_n) - beta * HQ[iy] - f[iy] * dHQdy[iy] 
            F[iy]     = D[iy] / (g*(H0+H[iy]))
            dFdy[iy]  = dDdy[iy] / (g*(H0+H[iy])) - (D[iy]*dHdy[iy]) / (g*((H0+H[iy])**2))
            G[iy]     = (k**2)*((U[iy]-c_n)**2 - g*(H0+H[iy]))                                    # ODE for v
            dGdy[iy]  = (k**2)*(2*dUdy[iy]*(U[iy]-c_n) - g*dHdy[iy])                              # ODE for v
            R[iy]     = (-1.0 / F[iy])*dFdy[iy]
            S[iy]     = ((F[iy] - (k**2)) + ((f[iy] * dFdy[iy] / F[iy]) - beta) / (U[iy] - c_n))
            Rv[iy]    = (-dGdy[iy]/(G[iy]*(U[iy]-c_n)))                                           # ODE for v
            Sv[iy]    = (F[iy]-(k**2))-(dHQdy[iy] - (HQ[iy]/G[iy])*dGdy[iy])/(U[iy]-c_n)          # ODE for v 
            Rs[iy]    = 0
            Ss[iy]    = (dHQdy[iy])/(U[iy]-c_n) -(k**2)
            
        if i == 5:
            f[iy]     = beta*y[iy]
            B[iy]     = (y[iy]**2)/(4.0)
            m         = 1
            U[iy]     = A[0]*np.cos(m*np.pi*y[iy]/(L))   
            H[iy]     = -(A[0]*(beta) / g)*((L/m*np.pi)**2)*((np.cos(m*np.pi*y[iy]/(L))+(L/(m*np.pi))*y[iy]*np.sin(m*np.pi*y[iy]/(L))))
            dUdy[iy]  = -A[0]*(m*np.pi/L)*np.sin(m*np.pi*y[iy]/(L))
            dHdy[iy]  = -(beta/(g))*y[iy]*A[0]*np.cos(m*np.pi*y[iy]/(L))
            HQ[iy]    = (beta)*y[iy] + A[0]*(m*np.pi/L)*np.sin(m*np.pi*y[iy]/(L))
            dHQdy[iy] =  beta + A[0]*((m*np.pi/L)**2)*np.cos(m*np.pi*y[iy]/(L)) 
            D[iy]     = (k**2) * ((U[iy] - c_n)**2) - f[iy] * HQ[iy]
            dDdy[iy]  = 2 * (k**2) * dUdy[iy] * (U[iy] - c_n) - beta * HQ[iy] - f[iy] * dHQdy[iy] 
            F[iy]     = D[iy] / (g*(H0+H[iy]))
            dFdy[iy]  = dDdy[iy] / (g*(H0+H[iy])) - (D[iy]*dHdy[iy]) / (g*((H0+H[iy])**2))
            G[iy]     = (k**2)*((U[iy]-c_n)**2 - g*(H0+H[iy]))                                    # ODE for v
            dGdy[iy]  = (k**2)*(2*dUdy[iy]*(U[iy]-c_n) - g*dHdy[iy])                              # ODE for v
            R[iy]     = (-1.0 / F[iy])*dFdy[iy]
            S[iy]     = ((F[iy] - (k**2)) + ((f[iy] * dFdy[iy] / F[iy]) - beta) / (U[iy] - c_n))
            Rv[iy]    = (-dGdy[iy]/(G[iy]*(U[iy]-c_n)))                                           # ODE for v
            Sv[iy]    = (F[iy]-(k**2))-(dHQdy[iy] - (HQ[iy]/G[iy])*dGdy[iy])/(U[iy]-c_n)          # ODE for v
            Rs[iy]    = 0
            Ss[iy]    = (dHQdy[iy])/(U[iy]-c_n) -(k**2)
        
 
    return R, S, U, H, dUdy, dHdy, HQ, dHQdy, D ,dDdy, F, dFdy, G, dGdy, f, Rv, Sv, Rs, Ss

In [12]:
#------------------------------------------------------------------------------------------------------------------------------
# Section 3.0 : Define an ODE solver that will use the previous functions to solve the ODE
#------------------------------------------------------------------------------------------------------------------------------
def ODE_SOLVER_REES():
    import sys
    import math
    import cmath
    import numpy as np
    import matplotlib.pyplot as plt
    import time
    
    start_time=time.time()
#------------------------------------------------------------------------------------------------------------------------------
# Section 3.1 : Set up the grid in y, allocate memory and define the constants
#------------------------------------------------------------------------------------------------------------------------------
    Levels    = 100                 # Number of levels
    Tolerance = 1e-7                # Tolerance for errors (see Section 3.4)
    beta      = 2.2*(10**-11)       
    g         = 0.0334              # Reduced gravity
    H0        = 60                  # Average thermocline depth (see Tanaka+Hibiya)
    c         = np.sqrt(g*H0) 
    C         = np.sqrt((2*beta/c)) 

    y  = np.zeros(Levels + 2)       # Allocate memory to y
    dy = 1.0 / float(Levels)        # float/float = float
    L = 10                          # Length of half the domain
    M = 2*L                         # Chosen so that y(-L)=Dom, y(+L)=-Dom
    for n in range(Levels + 2):
        y[n] = -L + M * n * dy  
    
#-------------------------------------------------------------------------------------------------------------------------------
# Section 3.2.0 : Set up the values for the wave speed and wave number that we will loop over 
#-------------------------------------------------------------------------------------------------------------------------------
    cs      = np.zeros( (1,1) , dtype = np.complex) # Allocate memory for wave speed
    #c_0_re  = -0.3                                 # Real part of the prescribed initial wave speed
    c_0_re  = -(beta*(L**2)/(np.pi**2))
    c_0_im  = 0.01                                  # Imaginary part of the prescribed wave speed
    #c_0_im  = 0.01                                 # Imaginary part of the prescribed wave speed
    c_0     = c_0_re + 1j * c_0_im                  # Starting point for the wave speed (Take an educated guess)
    d_c_inc = 0.01                                  # Increment that we will add to the previous wave speed after each loop
    d_c     = 1j * d_c_inc                          # Since c is complex we need to make the increment complex too
    k       = 0.2                                   # Prescribed choice of wavenumber
    #k       = abs(math.sqrt(5*(np.pi**2)/(4*(10**2))))
    d_k     = 0.01                                  # Similarly, we increment over the wavenumber after each loop
    
    SMALL_SIZE = 14                                 # Set the sizes for the text in plots
    MEDIUM_SIZE = 16
    BIGGER_SIZE = 22
    
#------------------------------------------------------------------------------------------------------------------------------
# Section 3.2.1: Print the user defined options
#-----------------------------------------------------------------------------------------------------------------------------
    with open('User_Def_Input.txt','w') as file_out:
        print("User Defined Inputs:"   ,file=file_out)
        print("Levels    = ",    Levels,file=file_out)
        print("Tolerance = ", Tolerance,file=file_out)
        print("beta      = ",      beta,file=file_out)
        print("g         = ",         g,file=file_out)
        print("H0        = ",        H0,file=file_out)
        print("c         = ",         c,file=file_out)
        print("C         = ",         C,file=file_out)
        print("y         = ",         y,file=file_out)
        #print("dy        = ",        dy,file=file_out)
        print("n         = ",         n,file=file_out)
        print("cs        = ",        cs,file=file_out)
        print("c_0_re    = ",    c_0_re,file=file_out)
        print("c_0_im    = ",    c_0_im,file=file_out)
        print("c_0       = ",       c_0,file=file_out)
        print("d_c_inc   = ",   d_c_inc,file=file_out)
        print("d_c       = ",       d_c,file=file_out)
        print("k         = ",         k,file=file_out)
        print("d_k       = ",       d_k,file=file_out)
        
#------------------------------------------------------------------------------------------------------------------------------
# Section 3.3 : Find R and S for c_0 (first choice of wave speed) and then integrate to find phi with boundary conditions
#------------------------------------------------------------------------------------------------------------------------------    
#------------------------------------------------------------------------------------------------------------------------------
# Section 3.3.1: Begin the iteration towards a solution for wavenumber in the outer loop 
#------------------------------------------------------------------------------------------------------------------------------
# The inner loop (see Section 3.3) involves all the hard work of finding the wave speed for a given wavenumber. The outer loop
# that we start here iterates over a range of wavenumbers so that we can then find (and plot) the growth rate k*c_{i}.
    
    Max_iter_1 = 1 # We will need more than one, but start off with this as a test
    #K = np.zeros(Max_iter_1) 
    for iter_outer in range(Max_iter_1):
        
#------------------------------------------------------------------------------------------------------------------------------
# Section 3.3.2: Solving for the first choice of wave speed
#------------------------------------------------------------------------------------------------------------------------------
# This section contains the basic structure for how we solve the problem. Begin by calculating P and Q for this choice of wave
# speed. Set the boudary condition and the integrate through the domain to find each phi. Then we calculate the error at the 
# upper boundary.
        
        R, S, U, H, dUdy, dHdy, HQ, dHQdy, D, dDdy, F, dFdy, G, dGdy, f, Rv, Sv, Rs, Ss = R_S_Rees(y, c_0 , k)  # Calculate R and S for the initial choice of wavespeed

        PHI = np.zeros(Levels + 2, dtype = np.complex) # Allocate memory for phi
        PHI[0] = 0.0                                   # Boundary condition
        PHI[1] = 1.0                                   # Sets the otherwise arbitrary amplitude

        #Integrate up through the domain
        for n in range(1,Levels + 1): # PHI[2] to PHI[Levels + 1]
            PHI[n+1] = INT_REES(PHI[n-1],PHI[n],R[n],S[n],y[n-1],y[n],y[n+1])

        #PHI[Levels] = 0.0
        # Calculate the error at the upper boundary
        PHI_NU_BC = PHI[Levels]     # PHI_NU_BC_0
        E_0 = PHI[Levels + 1] - PHI_NU_BC  # Upper boundary condition minus the step before it
        PHI_NU_BC_0 = PHI[Levels]

        
        # Print the outputs for the functions evaluated over y and at (k,c_0)
        with open('Output_c0.txt','w') as file_out:
            print("For the initial evaluation at c0, the outputs are:",file=file_out)
            print(""                     ,file=file_out)
            print("f = \n",f.real        ,file=file_out)
            print(""                     ,file=file_out)
            print("H = \n",H.real        ,file=file_out)
            print(""                     ,file=file_out)
            print("dHdy = \n",dHdy.real  ,file=file_out)
            print(""                     ,file=file_out)
            print("U = \n",U.real        ,file=file_out)
            print(""                     ,file=file_out)
            print("dUdy = \n",dUdy.real  ,file=file_out)
            print(""                     ,file=file_out)
            print("HQ = \n",HQ.real      ,file=file_out)
            print(""                     ,file=file_out)
            print("dHQdy = \n",dHQdy.real,file=file_out)
            print(""                     ,file=file_out)
            print("D = \n",D             ,file=file_out)
            print(""                     ,file=file_out)
            print("dDdy = \n",dDdy       ,file=file_out)
            print(""                     ,file=file_out)
            print("F = \n",F             ,file=file_out)
            print(""                     ,file=file_out)
            print("dFdy = \n",dFdy       ,file=file_out)
            print(""                     ,file=file_out)
            print("G = \n",G             ,file=file_out)
            print(""                     ,file=file_out)
            print("dGdy = \n",dGdy       ,file=file_out)
            print(""                     ,file=file_out)
            print("R = \n",R             ,file=file_out)
            print(""                     ,file=file_out)
            print("S = \n",S             ,file=file_out)
            print(""                     ,file=file_out)
            print("Rv = \n",Rv           ,file=file_out)
            print(""                     ,file=file_out)
            print("Sv = \n",Sv           ,file=file_out)
            print(""                     ,file=file_out)
            print("Rs = \n",Rs           ,file=file_out)
            print(""                     ,file=file_out)
            print("Ss = \n",Ss           ,file=file_out)
            print(""                     ,file=file_out)
            print("PHI = \n",PHI         ,file=file_out)
            print(""                     ,file=file_out)
            print("E_0 = \n",E_0         ,file=file_out)
        
#------------------------------------------------------------------------------------------------------------------------------
#Section 3.4 : Iterate towards a solution for the ODE over all the wave speed increments
#------------------------------------------------------------------------------------------------------------------------------  
#Now we have the same standard procedure as in the previous section but we are looping over this for different (increasingly
#accurate) wave speeds.

        #Start iterating towards a solution (i.e. increment c forwards)
        Max_iter_2 = 20      # Number of iterations,  try one as a test but will need more for an accurate c
        for iter_inner in range(Max_iter_2):
            c_1 = c_0  + d_c 

            R, S, U, H, dUdy, dHdy, HQ, dHQdy, D, dDdy, F, dFdy, G, dGdy, f, Rv, Sv, Rs, Ss = R_S_Rees(y,c_1,k) # Calculate P and Q for the new wave speed
            
            PHI = np.zeros(Levels + 2, dtype = np.complex) # Allocate memory for phi
            PHI[0] = 0.0                                   # Boundary condition
            PHI[1] = 1.0                                   # Sets the otherwise arbitrary amplitude

            #Integrate up through the domain
            for n in range(1,Levels + 1):
                PHI[n+1] = INT_REES(PHI[n-1],PHI[n],R[n],S[n],y[n-1],y[n],y[n+1]) # For c_1, find all the phi in the domain

            #PHI[Levels] = 0.0  Upper boundary condition
            # Calculate the error at the upper boundary
            PHI_NU_BC = PHI[Levels]
            E_1 = PHI[Levels + 1] - PHI_NU_BC
            PHI_NU_BC_1 = PHI[Levels]
            
            # To see why PHI blows up we check each term in the finite difference method
            denom = np.zeros(Levels+2, dtype = np.complex)
            num1  = np.zeros(Levels+2, dtype = np.complex)
            num2  = np.zeros(Levels+2, dtype = np.complex)
            for n in range(1,Levels+2):
                denom[n]   = 1.0/((y[n]-y[n-1])**2) + (R[n]/(2*(y[n]-y[n-1])))
                num2 [n]   = -(R[n] * PHI[n-1] / (2*(y[n]-y[n-1]))) + S[n] * PHI[n]   
                num1 [n]   = (-2.0 * PHI[n] + PHI[n-1]) / ((y[n]-y[n-1])**2) 
            
            # Print the functions evaluated over y and at (k,c_1)
            with open('Output_c1.txt','w') as file_out:
                print("Evaluated at c_1, the outputs are:",file=file_out)
                print(""                     ,file=file_out)
                print("f = \n",f.real        ,file=file_out)
                print(""                     ,file=file_out)
                print("H = \n",H.real        ,file=file_out)
                print(""                     ,file=file_out)
                print("dHdy = \n",dHdy.real  ,file=file_out)
                print(""                     ,file=file_out)
                print("U = \n",U.real        ,file=file_out)
                print(""                     ,file=file_out)
                print("dUdy = \n",dUdy.real  ,file=file_out)
                print(""                     ,file=file_out)
                print("HQ = \n",HQ.real      ,file=file_out)
                print(""                     ,file=file_out)
                print("dHQdy = \n",dHQdy.real,file=file_out)
                print(""                     ,file=file_out)
                print("D = \n",D             ,file=file_out)
                print(""                     ,file=file_out)
                print("dDdy = \n",dDdy       ,file=file_out)
                print(""                     ,file=file_out)
                print("F = \n",F             ,file=file_out)
                print(""                     ,file=file_out)
                print("dFdy = \n",dFdy       ,file=file_out)
                print(""                     ,file=file_out)
                print("G = \n",G             ,file=file_out)
                print(""                     ,file=file_out)
                print("dGdy = \n",dGdy       ,file=file_out)
                print(""                     ,file=file_out)
                print("R = \n",R             ,file=file_out)
                print(""                     ,file=file_out)
                print("S = \n",S             ,file=file_out)
                print(""                     ,file=file_out)
                print("Rv = \n",Rv           ,file=file_out)
                print(""                     ,file=file_out)
                print("Sv = \n",Sv           ,file=file_out)
                print(""                     ,file=file_out)
                print("Rs = \n",Rs           ,file=file_out)
                print(""                     ,file=file_out)
                print("Ss = \n",Ss           ,file=file_out)
                print(""                     ,file=file_out)
                print("PHI = \n",PHI         ,file=file_out)
                print(""                     ,file=file_out)
                print("E_1 = ",E_1           ,file=file_out)
                print(""                     ,file=file_out)
                print("denom = \n",denom     ,file=file_out)
                print(""                     ,file=file_out)
                print("num1 = \n",num1       ,file=file_out)
                print(""                     ,file=file_out)
                print("num2 = \n",num2       ,file=file_out)
                print(""                     ,file=file_out)
                print("PHI[Levels] = \n",PHI[Levels] ,file=file_out)
            
        # Print these values for each case (n) to determine why PHI blows up
        # Because this is an append ('a') we need to delete the file before running again
            for n in range(1,Levels+1):    
                with open('Debug_case.txt','a') as file_out:
                    print("Case number: n = ",n            ,file=file_out)
                    print(f'y({n-1})     = ' ,y[n-1]       ,file=file_out)
                    print(f'y({n})     = '   ,y[n]         ,file=file_out)
                    print(f'y({n+1})     = ' ,y[n+1]       ,file=file_out)
                    print(f'R({n})     = '   ,R.real[n]    ,file=file_out)
                   # print(f'Rv({n})    = '  ,Rv.real[n]   ,file=file_out)
                    print(f'S({n})     = '   ,S.real[n]    ,file=file_out)
                   # print(f'Sv({n})    = '  ,Sv.real[n]   ,file=file_out)
                    print(f'PHI({n-1})   = ' ,PHI.real[n-1],file=file_out)
                    print(f'PHI({n})   = '   ,PHI.real[n]  ,file=file_out)
                    print(f'denom({n}) = '   ,denom.real[n],file=file_out)
                    print(f'num1({n})  = '   ,num1.real[n] ,file=file_out)
                    print(f'num2({n})  = '   ,num2.real[n] ,file=file_out)
                    print(f'PHI({n+1})   = ' ,PHI.real[n+1],file=file_out)
                    print(""                               ,file=file_out)
            
# #------------------------------------------------------------------------------------------------------------------------------
# #Section 3.5 : Check for convergence and update the wave speed for the next loop
# #------------------------------------------------------------------------------------------------------------------------------
# #Here we check the convergence of the solution and update c via a shooting method and then store the final value of c in cs.
# #If the solution does not converge then we break the loop and stop iterating. 


#             if abs(E_1) < Tolerance:    
#                 if iter_inner % Levels == 0: # For the last iteration (when iter_inner = Levels)
#                     cs[0,0] = c_1            # Store the final value for c in cs
#                     c_0 = c_1                # Final update of c_0
#                     break

#                 # Calculate the updated values of c using a shooting method
#                 gradient = (c_1 - c_0)/(E_1 - E_0) 
#                 c_new = c_1 - gradient*E_1    

#                 if abs(E_1) < abs(E_0): # If absolute value of the error for c_1 is less than the old error for c_0 then,
#                     c_0 = c_1           # Save c_1 if it is better than c_0
#                     E_0 = E_1           # Take the least error so that we can be sure that the solution is converging

#                 c_1 = c_new             # Update c_1 in preperation for the next loop over wave speed
                
                

#                 # If the solution fails to converge for a given k then stop looping
#             if abs(E_1) > Tolerance :
#                 with open('Fail.txt','w') as file_out:
#                     print("Solution did not converge",file=file_out)
#                     #c_1 = c_0 + d_c
#                 break  

#----------------------------------------------------------------------------------------------------------------------------------------
# Section: New Shooting Method
#----------------------------------------------------------------------------------------------------------------------------------------
                
               
            if abs(PHI_NU_BC_1) < Tolerance:      # PHI_NU_BC_1 = PHI[Levels,c_1], PHI_NU_BC_0 = PHI[Levels,c_0]
                cs[0,0] = c_1                     # If there is more k then we need to store in the next index
                c_0 = c_1
                with open('Shooting Method 1.txt','w') as file_out:
                    print(f'Success',file=file_out)
                break

            if abs(PHI_NU_BC_1) > Tolerance and iter_inner < Max_iter_2:   # if PHI at the RHS boundary is not close to zero then
                gradient = (c_1 - c_0) / (PHI_NU_BC_1 - PHI_NU_BC_0)
                c_new = c_1 - gradient * PHI_NU_BC_1
                with open('Shooting Method 2.txt','a') as file_out:
                    print(f'iter_inner = ',iter_inner             ,file=file_out)
                    print(f''                                     ,file=file_out)
                    print(f'c_0.imag = ',c_0.imag                 ,file=file_out)
                    print(f''                                     ,file=file_out)
                    print(f'c_1.imag = ',c_1.imag                 ,file=file_out)
                    print(f''                                     ,file=file_out)
                    print(f'PHI_NU_BC_0 = ',PHI_NU_BC_0           ,file=file_out)
                    print(f''                                     ,file=file_out)
                    print(f'PHI_NU_BC_1 = ',PHI_NU_BC_1           ,file=file_out)
                    print(f''                                     ,file=file_out)
                    print(f'gradient.imag = ',gradient.imag       ,file=file_out)
                    print(f''                                     ,file=file_out)
                    print(f'c_new.imag = ',c_new.imag             ,file=file_out)
                    print(f''                                     ,file=file_out)
                
                if abs(PHI_NU_BC_1) < abs(PHI_NU_BC_0):
                    print(f'Step = {iter_inner}')
                    c_0 = c_1
                    PHI_NU_BC_0 = PHI_NU_BC_1
                    #c_1 = c_new
                    
                c_1 = c_new

            if iter_inner == Max_iter_2 - 1:
                print(f'Solution failed to converge in {Max_iter_2} steps')
                break
                
#------------------------------------------------------------------------------------------------------------------------------
# Section 3.5.1 : Plots for fixing k and varying c_{i}
#------------------------------------------------------------------------------------------------------------------------------
# In this section we fix a value for k and vary c_{i} (using Max_iter) whilst plotting the coefficients and solution of the ODE 
# We comment out the section 3.4/3.5 so that we don't waste time on unnecessary calculations
    
    
#         plt.figure(figsize=(9,6))

#         #ax1 = plt.subplot(221)
#         #ax1.plot(Rv.real,linewidth=2,label='R.real')
#         #ax1.plot(Rv.imag,linewidth=2,label='R.imag')
#         #ax1.plot([ind_min_mod_Dopp],[0],marker='o',markersize=4,color='red')
#         #ax1.plot([Levels - ind_min_mod_Dopp],[0],marker='o',markersize=4,color='red')
#         #ax1.set_xlim([0,Levels])
#         #ax1.set_title(f'R')
#         #plt.xticks((0,Levels/6.0,Levels/3.0,Levels/2.0,2*Levels/3.0,5*Levels/6.0,Levels),('-10','-6','-2','0','2','6','10'))
#         #ax1.grid()
#         #plt.legend()
#         #ax1.xaxis.set_ticklabels([])

#         ax2 = plt.subplot(211)
#         ax2.plot(Sv.real,linewidth=2,label='S.real')
#         ax2.plot(Sv.imag,linewidth=2,label='S.imag')
#         #ax1.plot([ind_min_mod_Dopp],[0],marker='o',markersize=4,color='red')
#         #ax1.plot([Levels - ind_min_mod_Dopp],[0],marker='o',markersize=4,color='red')
#         ax2.set_xlim([0,Levels])
#         ax2.set_title(f'S')
#         plt.xticks((0,Levels/6.0,Levels/3.0,Levels/2.0,2*Levels/3.0,5*Levels/6.0,Levels),('-10','-6','-2','0','2','6','10'))
#         ax2.grid()
#         #plt.legend()
#         #ax1.xaxis.set_ticklabels([])

#         ax3 = plt.subplot(223)
#         ax3.plot(PHI.real,linewidth=2)
#         ax3.set_xlim([0,Levels])
#         ax3.set_title(f'$Re(\phi)$')
#         ax3.set_xlabel(r"$\tilde{y}$")
#         plt.xticks((0,Levels/6.0,Levels/3.0,Levels/2.0,2*Levels/3.0,5*Levels/6.0,Levels),('-10','-6','-2','0','2','6','10'))
#         ax3.grid()
#         #ax2.xaxis.set_ticklabels([])

#         ax4 = plt.subplot(224)
#         ax4.plot(PHI.imag,linewidth=2)
#         ax4.set_xlim([0,Levels])
#         ax4.set_title(f'$Im(\phi)$')
#         ax4.set_xlabel(r"$\tilde{y}$")
#         plt.xticks((0,Levels/6.0,Levels/3.0,Levels/2.0,2*Levels/3.0,5*Levels/6.0,Levels),('-10','-6','-2','0','2','6','10'))
#         ax4.grid()
#         #ax2.xaxis.set_ticklabels([])
#         plt.suptitle(f'c_i={c_0.imag:.4f}')     # k is the title to 2dp
#         plt.rc('font',         size=SMALL_SIZE)    # controls default text sizes
#         plt.rc('axes',    titlesize=SMALL_SIZE)    # fontsize of the axes title
#         plt.rc('axes',   labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
#         plt.rc('xtick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
#         plt.rc('ytick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
#         plt.rc('legend',   fontsize=SMALL_SIZE)    # legend fontsize
#         plt.rc('figure', titlesize=BIGGER_SIZE)    # fontsize of the title

#         plt.subplots_adjust(hspace=0.5,wspace=0.5)
#         plt.savefig(f'Rs_vary_ci_%04d.png' %(iter_outer),dpi=300,bbox_inches='tight')
        
#         plt.show()
#         plt.close()
#         c_0 = c_0 + d_c
#-------------------------------------------------------------------------------------------------------------------------------
# Section 3.5.2 : Plots for fixing c_{i} and varying k
#-------------------------------------------------------------------------------------------------------------------------------
# In this section we fix a value for c_{i} and vary k (using Max_iter) whilst plotting the coefficients and solution of the ODE        
    
#         plt.figure(figsize=(9,6))

#         ax1 = plt.subplot(221)
#         ax1.plot(Rv.real,linewidth=2,label='R.real')
#         ax1.plot(Rv.imag,linewidth=2,label='R.imag')
#         #ax1.plot([ind_min_mod_Dopp],[0],marker='o',markersize=4,color='red')
#         #ax1.plot([Levels - ind_min_mod_Dopp],[0],marker='o',markersize=4,color='red')
#         ax1.set_xlim([0,Levels])
#         ax1.set_title(f'R')
#         plt.xticks((0,Levels/6.0,Levels/3.0,Levels/2.0,2*Levels/3.0,5*Levels/6.0,Levels),('-10','-6','-2','0','2','6','10'))
#         ax1.grid()
#         #plt.legend()
#         #ax1.xaxis.set_ticklabels([])

#         ax2 = plt.subplot(211)
#         ax2.plot(Ss.real,linewidth=2,label='S.real')
#         ax2.plot(Ss.imag,linewidth=2,label='S.imag')
#         #ax1.plot([ind_min_mod_Dopp],[0],marker='o',markersize=4,color='red')
#         #ax1.plot([Levels - ind_min_mod_Dopp],[0],marker='o',markersize=4,color='red')
#         ax2.set_xlim([0,Levels])
#         ax2.set_title(f'S')
#         plt.xticks((0,Levels/6.0,Levels/3.0,Levels/2.0,2*Levels/3.0,5*Levels/6.0,Levels),('-10','-6','-2','0','2','6','10'))
#         ax2.grid()
#         #plt.legend()
#         #ax1.xaxis.set_ticklabels([])

#         ax3 = plt.subplot(223)
#         ax3.plot(PHI.real,linewidth=2)
#         ax3.set_xlim([0,Levels])
#         ax3.set_title(f'$Re(\phi)$')
#         ax3.set_xlabel(r"$\tilde{y}$")
#         plt.xticks((0,Levels/6.0,Levels/3.0,Levels/2.0,2*Levels/3.0,5*Levels/6.0,Levels),('-10','-6','-2','0','2','6','10'))
#         ax3.grid()
#         #ax2.xaxis.set_ticklabels([])

#         ax4 = plt.subplot(224)
#         ax4.plot(PHI.imag,linewidth=2)
#         ax4.set_xlim([0,Levels])
#         ax4.set_title(f'$Im(\phi)$')
#         ax4.set_xlabel(r"$\tilde{y}$")
#         plt.xticks((0,Levels/6.0,Levels/3.0,Levels/2.0,2*Levels/3.0,5*Levels/6.0,Levels),('-10','-6','-2','0','2','6','10'))
#         ax4.grid()
#         #ax2.xaxis.set_ticklabels([])
#         plt.suptitle(f'k={k:.2f}')     # k is the title to 2dp
#         plt.rc('font',         size=SMALL_SIZE)    # controls default text sizes
#         plt.rc('axes',    titlesize=SMALL_SIZE)    # fontsize of the axes title
#         plt.rc('axes',   labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
#         plt.rc('xtick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
#         plt.rc('ytick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
#         plt.rc('legend',   fontsize=SMALL_SIZE)    # legend fontsize
#         plt.rc('figure', titlesize=BIGGER_SIZE)    # fontsize of the title

#         plt.subplots_adjust(hspace=0.5,wspace=0.5)
#         plt.savefig(f'Rs_vary_k_%04d.png' %(iter_outer),dpi=300,bbox_inches='tight')
        

#         plt.show()
#         plt.close()
#         k = k + d_k

#----------------------------------------------------------------------------------------------------------------------------------------
# Section 3.5.3: Fix k and vary c_{i} for abs(PHI)
#----------------------------------------------------------------------------------------------------------------------------------------
       
#         plt.plot(abs(PHI),linewidth=2)
#         plt.title(f'$|\phi|$, k={k}, ci={c_0.imag:.4f}')
#         plt.xlabel(r"$\tilde{y}$")
#         plt.xlim([0,Levels])
#         #plt.ylim([0,60])
#         plt.xticks((0,Levels/6.0,Levels/3.0,Levels/2.0,2*Levels/3.0,5*Levels/6.0,Levels),('-10','-6','-2','0','2','6','10'))
#         plt.grid()
        
#         plt.rc('font',         size=SMALL_SIZE)    # controls default text sizes
#         plt.rc('axes',    titlesize=SMALL_SIZE)    # fontsize of the axes title
#         plt.rc('axes',   labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
#         plt.rc('xtick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
#         plt.rc('ytick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
#         plt.rc('legend',   fontsize=SMALL_SIZE)    # legend fontsize
#         plt.rc('figure', titlesize=BIGGER_SIZE)    # fontsize of the title
#         plt.savefig(f'R_vary_k_abs_phi%04d.png' %(iter_outer),dpi=300,bbox_inches='tight')
        
#         plt.show()    
#         c_0 = c_0 + d_c

#----------------------------------------------------------------------------------------------------------------------------------------
# Section 3.5.4: Fix c_{i} and vary k for abs(PHI)
#----------------------------------------------------------------------------------------------------------------------------------------
        
#         plt.plot(abs(PHI),linewidth=2)
#         plt.title(f'$|\phi|$, k={k:.4f}, ci={c_0.imag:.4f}')
#         plt.xlabel(r"$\tilde{y}$")
#         plt.xlim([0,Levels])
#         #plt.ylim([0,60])
#         plt.xticks((0,Levels/6.0,Levels/3.0,Levels/2.0,2*Levels/3.0,5*Levels/6.0,Levels),('-10','-6','-2','0','2','6','10'))
#         plt.grid()
        
#         plt.rc('font',         size=SMALL_SIZE)    # controls default text sizes
#         plt.rc('axes',    titlesize=SMALL_SIZE)    # fontsize of the axes title
#         plt.rc('axes',   labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
#         plt.rc('xtick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
#         plt.rc('ytick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
#         plt.rc('legend',   fontsize=SMALL_SIZE)    # legend fontsize
#         plt.rc('figure', titlesize=BIGGER_SIZE)    # fontsize of the title
#         plt.savefig(f'Rv_vary_ci_abs_phi%04d.png' %(iter_outer),dpi=300,bbox_inches='tight')
        
#         plt.show()    
#         k = k + d_k
 
#------------------------------------------------------------------------------------------------------------------------------
# Section 3.6 : Update the wavenumber in preperation for the next loop
#------------------------------------------------------------------------------------------------------------------------------ 
    #K[iter_outer] = k   # K stores the wavenumbers that we iterate over
    #k = k + d_k         # Update k for the next loop

#------------------------------------------------------------------------------------------------------------------------------
# Section 4.0 : Set up for plots
#------------------------------------------------------------------------------------------------------------------------------
    X = 10                                       # Choose the individual plot we want (or comment out if we want a range)
    i = 0                                       # Profile case
    mod_Dopp = np.abs(U-c_0_re)                 # |U-c_{r}|
    min_mod_Dopp = np.amin(mod_Dopp)            # min|U-c_{r}| (approx where U=c)
    ind_min_mod_Dopp = np.argmin(mod_Dopp)      # Index where this occurs so we can plot a marker there
        
#-------------------------------------------------------------------------------------------------------------------------------
# Section 4.0.1 : Plots of the profiles and coefficients of the ODEs
#-------------------------------------------------------------------------------------------------------------------------------

#   X=0 [H0+H,dHdy,U,dUdy] | X=1 [HQ,dHQdy,D,dDdy] | X=2 [F,dFdy,G,dGdy] | X=3 [R,S] | X=4 [Rv,Sv] | X=5 [-(H0+H),U] 
#   X=6 [S,PHI] | X=7 [R,S,PHI]
     
    #for X in range(6,8):   # To plot and save multiple figures we can loop over all the plots 
    if X == 0:
        plt.figure(figsize=(12,4))

        ax1 = plt.subplot(221)
        ax1.plot(H0+H,linewidth=2)
        ax1.set_xlim([0,Levels])
        ax1.set_title('H')
        plt.xticks((0,20,40,60,80,100),('-10','-6','-2','2','6','10'))
        ax1.grid()
        #ax1.xaxis.set_ticklabels([])

        ax2 = plt.subplot(222)
        ax2.plot(dHdy,linewidth=2)
        ax2.set_xlim([0,Levels])
        ax2.set_title('dHdy')
        plt.xticks((0,20,40,60,80,100),('-10','-6','-2','2','6','10'))
        ax2.grid()
        #ax2.xaxis.set_ticklabels([])

        ax3 = plt.subplot(223)
        ax3.plot(U,linewidth=2)
        ax3.set_xlim([0,Levels])
        ax3.set_title('U')
        plt.xticks((0,20,40,60,80,100),('-10','-6','-2','2','6','10'))
        ax3.grid()
        #ax3.xaxis.set_ticklabels([])

        ax4 = plt.subplot(224)
        ax4.plot(dUdy,linewidth=2)
        ax4.set_xlim([0,Levels])
        ax4.set_title('dUdy')
        plt.xticks((0,20,40,60,80,100),('-10','-6','-2','2','6','10'))
        ax4.grid()
        #ax4.xaxis.set_ticklabels([])

        plt.rc('font',         size=SMALL_SIZE)    # controls default text sizes
        plt.rc('axes',    titlesize=SMALL_SIZE)    # fontsize of the axes title
        plt.rc('axes',   labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
        plt.rc('xtick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('ytick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('legend',   fontsize=SMALL_SIZE)    # legend fontsize
        plt.rc('figure', titlesize=BIGGER_SIZE)    # fontsize of the title

        plt.subplots_adjust(bottom=-0.5, right=1.2, left = 0, top=1)#,hspace=1)
        #plt.savefig(f'H,U,dHdy,dUdy_{i}.png',dpi=300,bbox_inches='tight')

        plt.show()

    if X == 1:
        plt.figure(figsize=(12,4))

        ax1 = plt.subplot(211)
        ax1.plot(HQ,linewidth=2)
        ax1.set_xlim([0,Levels])
        ax1.set_title('HQ')
        plt.xticks((0,20,40,60,80,100),('-10','-6','-2','2','6','10'))
        ax1.grid()
        #ax1.xaxis.set_ticklabels([])

        ax2 = plt.subplot(212)
        ax2.plot(dHQdy,linewidth=2)
        ax2.set_xlim([0,Levels])
        ax2.set_title('dHQdy')
        plt.xticks((0,20,40,60,80,100),('-10','-6','-2','2','6','10'))
        ax2.grid()
        #ax2.xaxis.set_ticklabels([])

        ax3 = plt.subplot(223)
        ax3.plot(D,linewidth=2)
        ax3.set_xlim([0,Levels])
        ax3.set_title('D')
        ax3.grid()
        #ax3.xaxis.set_ticklabels([])

        ax4 = plt.subplot(224)
        ax4.plot(dDdy,linewidth=2)
        ax4.set_xlim([0,Levels])
        ax4.set_title('dDdy')
        ax4.grid()
        #ax4.xaxis.set_ticklabels([])

        plt.rc('font',         size=SMALL_SIZE)    # controls default text sizes
        plt.rc('axes',    titlesize=SMALL_SIZE)    # fontsize of the axes title
        plt.rc('axes',   labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
        plt.rc('xtick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('ytick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('legend',   fontsize=SMALL_SIZE)    # legend fontsize
        plt.rc('figure', titlesize=BIGGER_SIZE)    # fontsize of the title

        plt.subplots_adjust(bottom=-0.5, right=1.2, left = 0, top=1)
        #plt.savefig(f'HQ,dHQdy_{i}.png',dpi=300,bbox_inches='tight')

        plt.show()
        plt.close()

    if X == 2:
        plt.figure(figsize=(12,4))

        ax1 = plt.subplot(221)
        ax1.plot(F.real,linewidth=2,label='F.real')
        ax1.plot(F.imag,linewidth=2,label='F.imag')
        ax1.set_xlim([0,Levels])
        ax1.set_title('F')
        plt.xticks((0,20,40,60,80,100),('-10','-6','-2','2','6','10'))
        ax1.grid()
        plt.legend()
        #ax1.xaxis.set_ticklabels([])


        ax2 = plt.subplot(222)
        ax2.plot(dFdy.real,linewidth=2,label='dFdy.real')
        ax2.plot(dFdy.imag,linewidth=2,label='dFdy.imag')
        ax2.set_xlim([0,Levels])
        ax2.set_title('dFdy')
        plt.xticks((0,20,40,60,80,100),('-10','-6','-2','2','6','10'))
        ax2.grid()
        plt.legend()
        #ax2.xaxis.set_ticklabels([])

        ax3 = plt.subplot(223)
        ax3.plot(G.real,linewidth=2,label='G.real')
        ax3.plot(G.imag,linewidth=2,label='G.imag')
        ax3.set_xlim([0,Levels])
        ax3.set_title('G')
        plt.xticks((0,20,40,60,80,100),('-10','-6','-2','2','6','10'))
        ax3.grid()
        plt.legend()
        #ax3.xaxis.set_ticklabels([])

        ax4 = plt.subplot(224)
        ax4.plot(dGdy.real,linewidth=2,label='dGdy.real')
        ax4.plot(dGdy.imag,linewidth=2,label='dGdy.imag')
        ax4.set_xlim([0,Levels])
        ax4.set_title('dGdy')
        plt.xticks((0,20,40,60,80,100),('-10','-6','-2','2','6','10'))
        ax4.grid()
        plt.legend()
        #ax4.xaxis.set_ticklabels([])

        plt.rc('font',         size=SMALL_SIZE)    # controls default text sizes
        plt.rc('axes',    titlesize=SMALL_SIZE)    # fontsize of the axes title
        plt.rc('axes',   labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
        plt.rc('xtick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('ytick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('legend',   fontsize=SMALL_SIZE)    # legend fontsize
        plt.rc('figure', titlesize=BIGGER_SIZE)    # fontsize of the title

        plt.subplots_adjust(bottom=-0.5, right=1.2, left = 0, top=1)
        #plt.savefig(f'F,dFdy,G,dGdy_{i}.png',dpi=300,bbox_inches='tight')

        plt.show()
        plt.close()

    if X == 3:
        plt.figure(figsize=(8,6))

        ax1 = plt.subplot(211)
        ax1.plot(R.real,linewidth=2,label='R.real')
        ax1.plot(R.imag,linewidth=2,label='R.imag')
        ax1.plot([ind_min_mod_Dopp],[0],marker='o',markersize=4,color='red')
        ax1.plot([Levels - ind_min_mod_Dopp],[0],marker='o',markersize=4,color='red')
        ax1.set_xlim([0,Levels])
        ax1.set_title('R')
        plt.xticks((0,20,40,60,80,100),('-10','-6','-2','2','6','10'))
        ax1.grid()
        #plt.legend()
        #ax1.xaxis.set_ticklabels([])

        ax2 = plt.subplot(212)
        ax2.plot(S.real,linewidth=2,label='S.real')
        ax2.plot(S.imag,linewidth=2,label='S.imag')
        ax2.set_xlim([0,Levels])
        ax2.set_title('S')
        ax2.set_xlabel(r"$\tilde{y}$")
        plt.xticks((0,20,40,60,80,100),('-10','-6','-2','2','6','10'))
        ax2.grid()
        #plt.legend()
        #ax2.xaxis.set_ticklabels([])

        plt.rc('font',         size=SMALL_SIZE)    # controls default text sizes
        plt.rc('axes',    titlesize=SMALL_SIZE)    # fontsize of the axes title
        plt.rc('axes',   labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
        plt.rc('xtick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('ytick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('legend',   fontsize=SMALL_SIZE)    # legend fontsize
        plt.rc('figure', titlesize=BIGGER_SIZE)    # fontsize of the title

        plt.subplots_adjust(bottom=0, right=1.2, left = 0, top=1,hspace=0.3)
        #plt.savefig(f'R,S.png',dpi=300,bbox_inches='tight')

        plt.show()
        plt.close()

    if X == 4:
        plt.figure(figsize=(8,6))

        ax1 = plt.subplot(211)
        ax1.plot(Rv.real,linewidth=2,label='Rv.real')
        #ax1.plot(Rv.imag,linewidth=2,label='Rv.imag')
        ax1.set_xlim([0,Levels])
        ax1.set_title('R')
        plt.xticks((0,20,40,60,80,100),('-10','-6','-2','2','6','10'))
        ax1.grid()
        #plt.legend()
        #ax1.xaxis.set_ticklabels([])


        ax2 = plt.subplot(212)
        ax2.plot(Sv.real,linewidth=2,label='Sv.real')
        #ax2.plot(Sv.imag,linewidth=2,label='Sv.imag')
        ax2.plot([ind_min_mod_Dopp],[0],marker='o',markersize=4,color='red')
        ax2.plot([Levels - ind_min_mod_Dopp],[0],marker='o',markersize=4,color='red')
        ax2.set_xlim([0,Levels])
        ax2.set_xlabel(r"$\tilde{y}$")
        ax2.set_title('S')
        plt.xticks((0,20,40,60,80,100),('-10','-6','-2','2','6','10'))
        ax2.grid()
        #plt.legend()
        #ax2.xaxis.set_ticklabels([])

        plt.rc('font',         size=SMALL_SIZE)    # controls default text sizes
        plt.rc('axes',    titlesize=SMALL_SIZE)    # fontsize of the axes title
        plt.rc('axes',   labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
        plt.rc('xtick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('ytick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('legend',   fontsize=SMALL_SIZE)    # legend fontsize
        plt.rc('figure', titlesize=BIGGER_SIZE)    # fontsize of the title

        plt.subplots_adjust(bottom=-0.5, right=1.2, left = 0, top=1)
        #plt.savefig(f'Pv,Qv.png',dpi=300,bbox_inches='tight')

        plt.show()
        plt.close()

    if X == 5:    
        plt.figure(figsize=(6,6))

        ax1 = plt.subplot(211)
        ax1.plot(-(H0+H),linewidth=2)
        ax1.set_xlim([20,Levels-20])
        ax1.set_title('H')
        plt.xticks((20,40,60,80),('-6','-2','2','6'))
        plt.yticks((-54.0,-56.0,-58.0,-60.0,-62.0,-64.0),(54.0,56.0,58.0,60.0,62.0,64.0))
        plt.ylabel('Depth (m)')
        ax1.grid()
        #ax1.xaxis.set_ticklabels([])


        ax2 = plt.subplot(212)
        ax2.plot(U,linewidth=2)
        ax2.set_xlim([20,Levels-20])
        ax2.set_title('U')
        ax2.set_xlabel(r"$\tilde{y}$")
        plt.xticks((0,20,40,60,80,100),('-10','-6','-2','2','6','10'))
        plt.ylabel('Zonal Velocity (m/s)')
        ax2.grid()
        #ax2.xaxis.set_ticklabels([])

        plt.rc('font',         size=SMALL_SIZE)    # controls default text sizes
        plt.rc('axes',    titlesize=SMALL_SIZE)    # fontsize of the axes title
        plt.rc('axes',   labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
        plt.rc('xtick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('ytick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('legend',   fontsize=SMALL_SIZE)    # legend fontsize
        plt.rc('figure', titlesize=BIGGER_SIZE)    # fontsize of the title

        plt.subplots_adjust(bottom=-0.5, right=1.2, left = 0, top=1,hspace=0.3)
        #plt.savefig(f'U_Presentation.png',dpi=300,bbox_inches='tight')

        plt.show()

    if X == 6:
        plt.figure(figsize=(6,6))

        ax1 = plt.subplot(211)
        ax1.plot(Ss.real,linewidth=2,label='S.real')
        ax1.plot(Ss.imag,linewidth=2,label='S.imag')
        #ax1.plot([ind_min_mod_Dopp],[0],marker='o',markersize=4,color='red')
        #ax1.plot([Levels - ind_min_mod_Dopp],[0],marker='o',markersize=4,color='red')
        ax1.set_xlim([0,Levels])
        ax1.set_title(f'S')
        ax1.set_xlabel(r"$\tilde{y}$")
        plt.xticks((0,Levels/6.0,Levels/3.0,Levels/2.0,2*Levels/3.0,5*Levels/6.0,Levels),('-10','-6','-2','0','2','6','10'))
        ax1.grid()
        plt.legend()
        #ax1.xaxis.set_ticklabels([])

        ax2 = plt.subplot(223)
        ax2.plot(PHI.real,linewidth=2)
        ax2.set_xlim([0,Levels])
        ax2.set_title(f'$Re(\phi)$')
        ax2.set_xlabel(r"$\tilde{y}$")
        plt.xticks((0,Levels/6.0,Levels/3.0,Levels/2.0,2*Levels/3.0,5*Levels/6.0,Levels),('-10','-6','-2','0','2','6','10'))
        ax2.grid()
        #ax2.xaxis.set_ticklabels([])

        ax3 = plt.subplot(224)
        ax3.plot(PHI.imag,linewidth=2)
        ax3.set_xlim([0,Levels])
        ax3.set_title(f'$Im(\phi)$')
        ax3.set_xlabel(r"$\tilde{y}$")
        plt.xticks((0,Levels/6.0,Levels/3.0,Levels/2.0,2*Levels/3.0,5*Levels/6.0,Levels),('-10','-6','-2','0','2','6','10'))
        ax3.grid()
        #ax2.xaxis.set_ticklabels([])

        plt.rc('font',         size=SMALL_SIZE)    # controls default text sizes
        plt.rc('axes',    titlesize=SMALL_SIZE)    # fontsize of the axes title
        plt.rc('axes',   labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
        plt.rc('xtick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('ytick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('legend',   fontsize=SMALL_SIZE)    # legend fontsize
        plt.rc('figure', titlesize=BIGGER_SIZE)    # fontsize of the title 

        plt.subplots_adjust(bottom=0, right=1.2, left = 0, top=1,hspace=0.5,wspace=0.5)
        #plt.savefig(f'Ss_PHI_k=035_cr=0_ci=01.png',dpi=300,bbox_inches='tight')

        plt.show()

    if X == 7:
        plt.figure(figsize=(9,6))

        ax1 = plt.subplot(221)
        ax1.plot(Rv.real,linewidth=2,label='R.real')
        ax1.plot(Rv.imag,linewidth=2,label='R.imag')
        #ax1.plot([ind_min_mod_Dopp],[0],marker='o',markersize=4,color='red')
        #ax1.plot([Levels - ind_min_mod_Dopp],[0],marker='o',markersize=4,color='red')
        ax1.set_xlim([0,Levels])
        ax1.set_title(f'R')
        ax1.set_xlabel(r"$\tilde{y}$")
        plt.xticks((0,Levels/6.0,Levels/3.0,Levels/2.0,2*Levels/3.0,5*Levels/6.0,Levels),('-10','-6','-2','0','2','6','10'))
        ax1.grid()
        #plt.legend()
        #ax1.xaxis.set_ticklabels([])

        ax2 = plt.subplot(222)
        ax2.plot(Sv.real,linewidth=2,label='S.real')
        ax2.plot(Sv.imag,linewidth=2,label='S.imag')
        #ax1.plot([ind_min_mod_Dopp],[0],marker='o',markersize=4,color='red')
        #ax1.plot([Levels - ind_min_mod_Dopp],[0],marker='o',markersize=4,color='red')
        ax2.set_xlim([0,Levels])
        ax2.set_title(f'S')
        ax2.set_xlabel(r"$\tilde{y}$")
        plt.xticks((0,Levels/6.0,Levels/3.0,Levels/2.0,2*Levels/3.0,5*Levels/6.0,Levels),('-10','-6','-2','0','2','6','10'))
        ax2.grid()
        #plt.legend()
        #ax1.xaxis.set_ticklabels([])

        ax3 = plt.subplot(223)
        ax3.plot(PHI.real,linewidth=2)
        ax3.set_xlim([0,Levels])
        ax3.set_title(f'$Re(\phi)$')
        ax3.set_xlabel(r"$\tilde{y}$")
        plt.xticks((0,Levels/6.0,Levels/3.0,Levels/2.0,2*Levels/3.0,5*Levels/6.0,Levels),('-10','-6','-2','0','2','6','10'))
        ax3.grid()
        #ax2.xaxis.set_ticklabels([])

        ax4 = plt.subplot(224)
        ax4.plot(PHI.imag,linewidth=2)
        ax4.set_xlim([0,Levels])
        ax4.set_title(f'$Im(\phi)$')
        ax4.set_xlabel(r"$\tilde{y}$")
        plt.xticks((0,Levels/6.0,Levels/3.0,Levels/2.0,2*Levels/3.0,5*Levels/6.0,Levels),('-10','-6','-2','0','2','6','10'))
        ax4.grid()
        #ax2.xaxis.set_ticklabels([])

        plt.rc('font',         size=SMALL_SIZE)    # controls default text sizes
        plt.rc('axes',    titlesize=SMALL_SIZE)    # fontsize of the axes title
        plt.rc('axes',   labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
        plt.rc('xtick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('ytick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('legend',   fontsize=SMALL_SIZE)    # legend fontsize
        plt.rc('figure', titlesize=BIGGER_SIZE)    # fontsize of the title

        plt.subplots_adjust(bottom=0, right=1.2, left = 0, top=1,hspace=0.5,wspace=0.5)
        #plt.savefig(f'Pv_Qv_PHI_k=SWC_cr=0_ci=001.png',dpi=300,bbox_inches='tight')

        plt.show()
        
    if X == 8:
        plt.plot(abs(PHI),linewidth=2)
        plt.title(f'$|\phi|$')
        plt.xlabel(r"$\tilde{y}$")
        plt.xlim([0,Levels])
        plt.xticks((0,Levels/6.0,Levels/3.0,Levels/2.0,2*Levels/3.0,5*Levels/6.0,Levels),('-10','-6','-2','0','2','6','10'))
        plt.grid()
        
        plt.rc('font',         size=SMALL_SIZE)    # controls default text sizes
        plt.rc('axes',    titlesize=SMALL_SIZE)    # fontsize of the axes title
        plt.rc('axes',   labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
        plt.rc('xtick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('ytick',   labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('legend',   fontsize=SMALL_SIZE)    # legend fontsize
        plt.rc('figure', titlesize=BIGGER_SIZE)    # fontsize of the title
        plt.show()
    
#--------------------------------------------------------------------------------------------------------------------------
# Section 4.1 : Plot of the growth rate
#--------------------------------------------------------------------------------------------------------------------------
 
    #plt.plot(K,K*c_0.imag,linewidth=2)
    #plt.title('Growth Rate')
    #plt.xlabel('wavenumber k')
    #plt.ylabel('')
    #plt.grid()
    #plt.show()
        
#--------------------------------------------------------------------------------------------------------------------------
# Section 4.2 : Plot of the real part of the phase speed
#--------------------------------------------------------------------------------------------------------------------------

    #plt.plot(K,K*c_0.real,linewidth=2)
    #plt.title('Real part of phase speed')
    #plt.grid()
    #plt.show()
    
#--------------------------------------------------------------------------------------------------------------------------    
# Section 4.3 : Plot out the levels where the ODEs are singular
#--------------------------------------------------------------------------------------------------------------------------

#    if (some condition to determine which ODE we are considering):
#        # ODE for eta
#        asing1 = math.sqrt(f*HQ/(k**2))  # U-c = 0 or (U-c)^{2}=f*HQ/k^{2} \implies F = 0 
#        asingular = [0,-asing1,asing1]   # Singular points in terms of U-c
        
#    else:
#        # ODE for v
#        asingular=[0]    # Singular point for the ODE in v
        
#        n_kappa = len(K) # Number of wavenumbers that we have iterated over
#        Ones = np.ones(n_kappa)
#        Asing = asingular*Ones # Singular points in ones, to fill the plot
#        ysing = Asing/K + c_n # Not sure what is happening here ? 
#        
#        plt.plot(K,ysing.real) # plots the levels where the ODEs are singular

#--------------------------------------------------------------------------------------------------------------------------
# Section 5.0 : Time taken for the code to run
#--------------------------------------------------------------------------------------------------------------------------
    print("--- %s seconds ---" % (time.time() - start_time))
ODE_SOLVER_REES()

Step = 0
Step = 1
Step = 2
Step = 3
Step = 4
Step = 5
Step = 6
Step = 7
Step = 8
Step = 9
Step = 10
Step = 11
Step = 12
Step = 13
Step = 14
Step = 15
Step = 16
Step = 17
Step = 18
Solution failed to converge in 20 steps
--- 8.40305757522583 seconds ---
